## Week 3 Assignment

In this assignment I will be exploring an clustering neighbourhoods in Toronto.

### Part 1

Import neighbourhood data from Wikipedia and create a dataframe of postal code, borough and neighbourhood.

In [2]:
#Import required libraries
!pip install bs4
!pip install pandas
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
#Create soup object for Wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result=requests.get(url).content
soup=BeautifulSoup(result)
#soup

In [4]:
#Extract relevant data and convert to text string.
#Format string in suitable format to create a list. 
table=soup.find('table')
Text=table.text
Text
Text1=Text.replace('\n','_').replace('____','_').replace('___','_').replace('__','_').replace('(',',').replace(')','').replace('Not assigned','Not assigned, ')
Text1

#Create list where each element is one post code.
List=list(Text1.split("_"))
del(List[0])
del(List[-1])
#List

In [5]:
#Loop through List to add a space after each post code.
i=0

while i < len(List):
    a=List[i]
    b=a[0:3] + ',' + a[3:]
    List[i]=b  
    i=i+1

#List

In [6]:
#Split List into 3 lists for Postcode, neighbourhood and boroughs
list1, list2, list3 = [x.split(',')[0] for x in List], [x.split(',')[1] for x in List] , [x.split(',')[2] for x in List]
print('PostalCodes: ',len(list1))
print('Boroughs: ',len(list2))
print('Neighbourhoods: ',len(list3))

PostalCodes:  180
Boroughs:  180
Neighbourhoods:  180


In [7]:
#Create Pandas dataframe.
df = pd.DataFrame(list(zip(list1, list2, list3)),
               columns =['PostalCode', 'Borough', 'Neighbourhood'])

df['Neighbourhood']=df['Neighbourhood'].str.replace(' / ',', ')
df.shape

(180, 3)

In [8]:
#Drop rows with and not Assigned Boroughs.
df=df[df['Borough'] != 'Not assigned' ]
df.reset_index(drop=True, inplace=True)
df.shape

(103, 3)

In [9]:
#Check no 'Not Assigned' or missing Neighbourhoods
NANeighbourhood=df[df['Neighbourhood'] == 'Not assigned']
MissNeighbourhood=df[df['Neighbourhood'] == '']
NANeighbourhood 
MissNeighbourhood
#No observations

,PostalCode,Borough,Neighbourhood


In [10]:
#Check no duplicated postal codes.
dupPC = df[df['PostalCode'].duplicated()]
dupPC
#No observations

,PostalCode,Borough,Neighbourhood


In [11]:
#Demonstrate number of rows and columns in dataframe
df.shape

(103, 3)

### Part 2

Add latitude and longitude coordinates for each Postal Code

In [12]:
#Get latitude and longitude coordinates for each post code.
        #*****Note: Unable to get geocoder to work - using csv file instead*****

#!pip install geocoder
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

#postal_code=df['PostalCode']

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]


#Read in csv file, convert to dataframe and adjust as required for merge
csv='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
df_csv = pd.read_csv(csv)
df_csv.rename({'Postal Code': 'PostalCode'}, axis=1, inplace=True) #Rename as per df dataframe.
df_csv

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [13]:
#Merge original dataframe and csv dataframe to map lat and long to original data.
df_out=pd.merge(
    df,
    df_csv,
    how="inner",
    on='PostalCode',
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
#    suffixes=None,
    copy=True,
    indicator=False,
    validate=None,
)
df_out.shape

(103, 5)

In [14]:
#Check for missing co-ordinates
df_out.info()

#No missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PostalCode     103 non-null    object 
 1   Borough        103 non-null    object 
 2   Neighbourhood  103 non-null    object 
 3   Latitude       103 non-null    float64
 4   Longitude      103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [15]:
#Demonstrate final dataframe
df_out

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### Part 3

In Part 3 I will explore and cluster neighbourhoods in Toronto. I will start by visualising the data by plotting each neighbourhood on a map. I will then choose a specific area(s) to focus on.

In [17]:
#Install libraries and assign Toronto coordinates
!pip install folium
import folium 

Tor_lat=43.6532
Tor_long=-79.3832


In [18]:
Tor_map=folium.Map(location=[Tor_lat, Tor_long],zoom_start=11)

for lat,long,pc,bor in zip(df_out['Latitude'], df_out['Longitude'], df_out['PostalCode'], df_out['Borough']):
    label=pc+' '+bor
    folium.CircleMarker(location=[lat, long],radius=2,weight=5).add_child(folium.Popup(label)).add_to(Tor_map)
    
Tor_map

In [19]:
#Check number of post codes for each borough 
df_out['Borough'].value_counts()

North York                                                      24
Downtown Toronto                                                17
Scarborough                                                     17
Etobicoke                                                       11
Central Toronto                                                  9
West Toronto                                                     6
York                                                             5
East York                                                        4
East Toronto                                                     4
EtobicokeNorthwest                                               1
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
MississaugaCanada Post Gateway Processing Centre                 1
Queen's Park                                                     1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                         

#### Note:
I will base the rest of the neighbourhood clustering/exploring analysis on the following four boroughs: 
Downtown Toronto, Central Toronto, West Toronto, and East Toronto. 

I will also make some assumptions and include 'East TorontoBusiness reply mail Processing Centre969 Eastern' in the 'East Toronto' borough, and the 'Downtown TorontoStn A PO Boxes25 The Esplanade' in the 'Downtown Toronto' borough.

In [20]:
#Change Borough values as described above and confirm that the change was effective
df_out["Borough"].replace({"East TorontoBusiness reply mail Processing Centre969 Eastern": "East Toronto", "Downtown TorontoStn A PO Boxes25 The Esplanade": "Downtown Toronto"}, inplace=True)
df_out['Borough'].value_counts()

North York                                          24
Downtown Toronto                                    18
Scarborough                                         17
Etobicoke                                           11
Central Toronto                                      9
West Toronto                                         6
York                                                 5
East Toronto                                         5
East York                                            4
EtobicokeNorthwest                                   1
MississaugaCanada Post Gateway Processing Centre     1
Queen's Park                                         1
East YorkEast Toronto                                1
Name: Borough, dtype: int64

In [21]:
#Subset dataframe to extract specific boroughs noted above
df_sub=df_out.loc[df_out['Borough'].isin(['Downtown Toronto','Central Toronto','West Toronto','East Toronto'])].reset_index(drop=True)
df_sub['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [22]:
#Re-plot neighbourhoods from subsetted dataframe
sub_map=folium.Map(location=[Tor_lat, Tor_long],zoom_start=12)

for lat,long,pc,bor in zip(df_sub['Latitude'], df_sub['Longitude'], df_sub['PostalCode'], df_sub['Borough']):
    label=pc+' '+bor
    folium.CircleMarker(location=[lat, long],radius=2,weight=5).add_child(folium.Popup(label)).add_to(sub_map)
    
sub_map

In the following steps I will call the FourSquare API to gather information about venues in proximity to each of the above neighbourhoods

In [23]:
CLIENT_ID = 'H41B2EZTCCABXAN4W2VQMMEXVWOF2GYP2MZ3QLLRVSLIQSQF' # your Foursquare ID
CLIENT_SECRET = 'YOZHRR2KQROR0JT5PSSGPMNHIRSQDU2HEYB5UWUOPP1ODRTE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        API_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(API_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# type your answer here

toronto_venues=getNearbyVenues(df_sub['Neighbourhood'], df_sub['Latitude'], df_sub['Longitude'])
toronto_venues

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
...,...,...,...,...,...,...,...
1574,Enclave of M4L,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1575,Enclave of M4L,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1576,Enclave of M4L,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1577,Enclave of M4L,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


In [24]:
df_to_merge=df_sub[['Neighbourhood','Borough']].copy()
df_to_merge

toronto_venues=pd.merge(
    toronto_venues,
    df_to_merge,
    how='left',
    on='Neighbourhood',
    left_index=False,
    right_index=False,
    sort=True,
    copy=True,
    indicator=False,
    validate=None,
)
toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough
0,Berczy Park,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store,Downtown Toronto
1,Berczy Park,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant,Downtown Toronto
2,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto
3,Berczy Park,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum,Downtown Toronto
4,Berczy Park,43.644771,-79.373306,Goose Island Brewhouse,43.647329,-79.373541,Beer Bar,Downtown Toronto
...,...,...,...,...,...,...,...,...
1574,"University of Toronto, Harbord",43.662696,-79.400049,Magic Noodle,43.662728,-79.403602,Noodle House,Downtown Toronto
1575,"University of Toronto, Harbord",43.662696,-79.400049,Smoke's Poutinerie/Weinerie,43.658259,-79.400136,Poutine Place,Downtown Toronto
1576,"University of Toronto, Harbord",43.662696,-79.400049,The Beer Store,43.665385,-79.403477,Beer Store,Downtown Toronto
1577,"University of Toronto, Harbord",43.662696,-79.400049,Cafe On Harbord,43.662903,-79.403564,Café,Downtown Toronto


In [25]:
#How many venues are in each Neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough
Neighbourhood,,,,,,,
Berczy Park,59,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64,64
Christie,15,15,15,15,15,15,15
Church and Wellesley,78,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100
Davisville,37,37,37,37,37,37,37
Davisville North,8,8,8,8,8,8,8


In [27]:
toronto_venues.groupby('Venue Category').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Borough
Venue Category,,,,,,,
Adult Boutique,1,1,1,1,1,1,1
Airport,1,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
Video Game Store,2,2,2,2,2,2,2
Vietnamese Restaurant,7,7,7,7,7,7,7
Wine Bar,9,9,9,9,9,9,9
